In [ ]:
!pip install ultralytics

In [ ]:
!ls -l ../input/road-damage-dataset-potholes-cracks-and-manholes/data

In [ ]:
# Cella 1: Import e configurazione
import os
import random
import time
import math
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import shutil
import yaml
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import KFold
from ultralytics import YOLO
from tqdm import tqdm
import wandb

# Riproducibilità
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Configurazione
PROJECT = "pothole-detector-NatureSR"
ENTITY = "pothole-detector"
NUM_FOLDS = 4
IOU_THRESHOLD = 0.7
CONF_THRESHOLD = 0.25
INPUT_SIZE = 640

# Parametri training
CONFIG = {
    "batch_size": 16,
    "learning_rate": 0.01,
    "weight_decay": 0.0005,
    "momentum": 0.937,
    "epochs": 100,
    "patience": 10,
}

CLASS_NAMES = ["pothole", "crack", "manhole"]

# Login wandb
wandb.login(key="key")
print("✓ Setup completato")

In [ ]:
# Cella 2: Preparazione dataset e fold
image_dir = "/kaggle/input/road-damage-dataset-potholes-cracks-and-manholes/data/images"
label_dir = "/kaggle/input/road-damage-dataset-potholes-cracks-and-manholes/data/labels"

# Lista file
image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
random.shuffle(image_files)
print(f"Dataset: {len(image_files)} immagini")

# Crea fold
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=0)
fold_splits = []

for fold_idx, (train_idx, val_idx) in enumerate(kf.split(image_files)):
    base_dir = f"runs/fold{fold_idx}"
    
    # Configurazione YAML per YOLO
    data_config = {
        'path': os.path.abspath(base_dir),
        'train': 'images/train',
        'val': 'images/val',
        'nc': 3,
        'names': CLASS_NAMES,
    }
    
    # Creazione directory
    for sub in ['images/train', 'images/val', 'labels/train', 'labels/val']:
        os.makedirs(os.path.join(base_dir, sub), exist_ok=True)
    
    # Copia file train
    train_files = [image_files[i] for i in train_idx]
    val_files = [image_files[i] for i in val_idx]
    
    for phase, files in zip(['train', 'val'], [train_files, val_files]):
        for img_file in files:
            lbl_file = img_file.rsplit('.', 1)[0] + '.txt'
            
            # Copia immagine
            shutil.copy(
                os.path.join(image_dir, img_file),
                os.path.join(base_dir, f'images/{phase}/{img_file}')
            )
            
            # Copia label se esiste
            src_lbl = os.path.join(label_dir, lbl_file)
            dst_lbl = os.path.join(base_dir, f'labels/{phase}/{lbl_file}')
            if os.path.exists(src_lbl):
                shutil.copy(src_lbl, dst_lbl)
    
    # Salva YAML
    yaml_path = os.path.join(base_dir, 'data.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(data_config, f)
    
    fold_splits.append({
        'train': train_files,
        'val': val_files,
        'yaml_path': yaml_path
    })
    
    print(f"Fold {fold_idx}: {len(train_files)} train, {len(val_files)} val")

print(f"✓ {NUM_FOLDS} fold creati")

In [ ]:
# Cella 3: Funzioni di training e metriche
def calculate_metrics(model, data_yaml, conf_threshold=CONF_THRESHOLD):
    """Calcola mAP e metriche per classe"""
    try:
        # Validazione con YOLO
        results = model.val(
            data=data_yaml,
            conf=conf_threshold,
            iou=IOU_THRESHOLD,
            save_json=True,
            verbose=False
        )
        
        metrics = results.box
        
        # Metriche per classe
        per_class = {}
        for i, name in enumerate(CLASS_NAMES):
            # Calcola F1 per classe
            p = float(metrics.p[i]) if i < len(metrics.p) else 0.0
            r = float(metrics.r[i]) if i < len(metrics.r) else 0.0
            f1 = 2*p*r/(p+r) if (p+r)>0 else 0.0
            
            per_class[name] = {
                'precision': p,
                'recall': r,
                'f1': f1
            }
        
        # Metriche globali
        precision = float(metrics.mp)
        recall = float(metrics.mr)
        f1 = 2*precision*recall/(precision+recall) if (precision+recall)>0 else 0.0
        
        return {
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'mAP50': float(metrics.map50),
            'mAP50-95': float(metrics.map),
            'per_class': per_class
        }
    
    except Exception as e:
        print(f"⚠️ Errore calcolo metriche: {e}")
        return {
            'precision': 0.0,
            'recall': 0.0,
            'f1': 0.0,
            'mAP50': 0.0,
            'mAP50-95': 0.0,
            'per_class': {name: {'precision': 0.0, 'recall': 0.0, 'f1': 0.0} for name in CLASS_NAMES}
        }

print("✓ Funzioni training definite")

In [ ]:
# Cella 4: Training singolo fold
def train_fold(fold_idx, fold_splits, config):
    """Training di un singolo fold"""
    print(f"\n{'='*60}")
    print(f"FOLD {fold_idx}")
    print(f"{'='*60}")
    
    # Inizializza modello
    model = YOLO("yolov8n.pt")
    
    yaml_path = fold_splits[fold_idx]['yaml_path']
    
    start_time = time.time()
    
    # Training con YOLO
    print(f"🔄 Training fold {fold_idx}...")
    results = model.train(
        data=yaml_path,
        epochs=config['epochs'],
        batch=config['batch_size'],
        imgsz=INPUT_SIZE,
        patience=config['patience'],
        device=0,
        amp=True,
        seed=0,
        deterministic=True,
        project=f"training_fold{fold_idx}",
        name="run",
        exist_ok=True,
        val=True,
        save=True,
        plots=False,
        verbose=False,
        workers=4,
    )
    
    training_time = time.time() - start_time
    
    # Determina epoche effettive
    actual_epochs = config['epochs']
    results_csv = f"training_fold{fold_idx}/run/results.csv"
    if os.path.exists(results_csv):
        actual_epochs = len(pd.read_csv(results_csv))
    
    # Carica best model per validazione finale
    best_model_path = f"training_fold{fold_idx}/run/weights/best.pt"
    if os.path.exists(best_model_path):
        model = YOLO(best_model_path)
    
    # Calcola metriche finali
    final_metrics = calculate_metrics(model, yaml_path)
    
    # Salva history (da results.csv di YOLO)
    history = []
    if os.path.exists(results_csv):
        df_results = pd.read_csv(results_csv)
        df_results = df_results.rename(columns=lambda x: x.strip())
        
        for idx, row in df_results.iterrows():
            epoch_data = {
                'epoch': idx + 1,
                'time': training_time * (idx + 1) / len(df_results),
            }
            
            # Mappa colonne YOLO a nostro formato
            column_mapping = {
                'train/box_loss': 'train/box_loss',
                'train/cls_loss': 'train/cls_loss',
                'val/box_loss': 'val/box_loss',
                'val/cls_loss': 'val/cls_loss',
                'metrics/precision(B)': 'metrics/precision(B)',
                'metrics/recall(B)': 'metrics/recall(B)',
                'metrics/mAP50(B)': 'metrics/mAP50(B)',
                'metrics/mAP50-95(B)': 'metrics/mAP50-95(B)',
            }
            
            for yolo_col, our_col in column_mapping.items():
                if yolo_col in df_results.columns:
                    epoch_data[our_col] = float(row[yolo_col])
            
            # Calcola train/total_loss e val/total_loss se non presenti
            if 'train/box_loss' in epoch_data and 'train/cls_loss' in epoch_data:
                epoch_data['train/total_loss'] = epoch_data['train/box_loss'] + epoch_data['train/cls_loss']
            
            if 'val/box_loss' in epoch_data and 'val/cls_loss' in epoch_data:
                epoch_data['val/total_loss'] = epoch_data['val/box_loss'] + epoch_data['val/cls_loss']
            
            # Learning rate (YOLO usa cosine annealing)
            epoch_data['lr/pg0'] = config['learning_rate'] * (1 + np.cos(np.pi * idx / len(df_results))) / 2
            
            history.append(epoch_data)
    
    # Salva results.csv
    df = pd.DataFrame(history)
    df.to_csv(f'fold{fold_idx}_results.csv', index=False)
    
    # Prepara dati per .pt
    class_logs = {}
    for i, name in enumerate(CLASS_NAMES):
        class_logs[f'fold{fold_idx}/precision_{name}'] = final_metrics['per_class'][name]['precision']
        class_logs[f'fold{fold_idx}/recall_{name}'] = final_metrics['per_class'][name]['recall']
        class_logs[f'fold{fold_idx}/f1_{name}'] = final_metrics['per_class'][name]['f1']
    
    class_logs.update({
        f'fold{fold_idx}/precision_mean': final_metrics['precision'],
        f'fold{fold_idx}/recall_mean': final_metrics['recall'],
        f'fold{fold_idx}/f1_mean': final_metrics['f1'],
        f'fold{fold_idx}/mAP50_mean': final_metrics['mAP50'],
        f'fold{fold_idx}/mAP_mean': final_metrics['mAP50-95'],
    })
    
    # Salva .pt
    torch.save({
        'fold': fold_idx,
        'class_logs': class_logs,
        'status': 'success',
        'training_time': training_time
    }, f'fold{fold_idx}_results.pt')
    
    # Copia best.pt
    if os.path.exists(best_model_path):
        shutil.copy(best_model_path, f'fold{fold_idx}_best.pt')
    
    print(f"\n✓ Fold {fold_idx} completato:")
    print(f"  Precision: {final_metrics['precision']:.4f}")
    print(f"  Recall: {final_metrics['recall']:.4f}")
    print(f"  mAP@0.5: {final_metrics['mAP50']:.4f}")
    print(f"  mAP@0.5:0.95: {final_metrics['mAP50-95']:.4f}")
    print(f"  Training time: {training_time/60:.1f} min")
    
    return final_metrics, training_time

print("✓ Funzione train_fold definita")

In [ ]:
# Cella 5: Cross-validation completa
def cross_validation():
    """Esegue cross-validation su tutti i fold"""
    print(f"\n🚀 Cross-Validation con {NUM_FOLDS} fold")
    
    # ✅ Inizializza wandb SUBITO all'inizio
    run = wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"YOLOv8n_cv_{NUM_FOLDS}fold",
        config=CONFIG
    )
    run_id = wandb.run.id
    
    all_results = []
    total_start = time.time()
    
    # Training di tutti i fold
    for fold_idx in range(NUM_FOLDS):
        try:
            metrics, train_time = train_fold(fold_idx, fold_splits, CONFIG)
            all_results.append({
                'fold': fold_idx,
                'precision': metrics['precision'],
                'recall': metrics['recall'],
                'mAP50': metrics['mAP50'],
                'mAP50-95': metrics['mAP50-95'],
                'training_time_min': train_time/60
            })
        except Exception as e:
            print(f"❌ Errore fold {fold_idx}: {e}")
            import traceback
            traceback.print_exc()
    
    total_time = time.time() - total_start
    
    if not all_results:
        print("❌ Nessun fold completato")
        wandb.finish()
        return False
    
    # Statistiche aggregate
    df = pd.DataFrame(all_results)
    summary = {
        'successful_folds': len(all_results),
        'total_time_hours': total_time/3600,
        'precision_mean': df['precision'].mean(),
        'precision_std': df['precision'].std(),
        'recall_mean': df['recall'].mean(),
        'recall_std': df['recall'].std(),
        'mAP50_mean': df['mAP50'].mean(),
        'mAP50_std': df['mAP50'].std(),
        'mAP50-95_mean': df['mAP50-95'].mean(),
        'mAP50-95_std': df['mAP50-95'].std(),
    }
    
    # Salva summary
    df.to_csv('cv_summary.csv', index=False)
    
    print(f"\n{'='*60}")
    print("RISULTATI FINALI")
    print(f"{'='*60}")
    print(f"Fold completati: {len(all_results)}/{NUM_FOLDS}")
    print(f"Tempo totale: {total_time/3600:.2f} ore")
    print(f"\nPrecision: {summary['precision_mean']:.4f} ± {summary['precision_std']:.4f}")
    print(f"Recall: {summary['recall_mean']:.4f} ± {summary['recall_std']:.4f}")
    print(f"mAP@0.5: {summary['mAP50_mean']:.4f} ± {summary['mAP50_std']:.4f}")
    print(f"mAP@0.5:0.95: {summary['mAP50-95_mean']:.4f} ± {summary['mAP50-95_std']:.4f}")
    
    # Log metriche aggregate nello STESSO run
    wandb.log({
        'cv/precision_mean': summary['precision_mean'],
        'cv/recall_mean': summary['recall_mean'],
        'cv/mAP50_mean': summary['mAP50_mean'],
        'cv/mAP50-95_mean': summary['mAP50-95_mean'],
        'experiment/successful_folds': len(all_results),
        'experiment/total_time_hours': total_time/3600
    })
    
    # Log metriche e crea artifact per ogni fold
    for fold_idx in range(NUM_FOLDS):
        pt_path = f'fold{fold_idx}_results.pt'
        csv_path = f'fold{fold_idx}_results.csv'
        pth_path = f'fold{fold_idx}_best.pt'
        
        if os.path.exists(pt_path):
            data = torch.load(pt_path)
            if data.get('status') == 'success':
                wandb.log(data['class_logs'])
                
                # Crea artifact per questo fold
                artifact = wandb.Artifact(f'fold{fold_idx}_results', type='model_results')
                artifact.add_file(pt_path)
                print(f"✓ Fold {fold_idx}: Aggiunto {pt_path}")
                
                if os.path.exists(csv_path):
                    artifact.add_file(csv_path)
                    print(f"✓ Fold {fold_idx}: Aggiunto {csv_path}")
                
                if os.path.exists(pth_path):
                    artifact.add_file(pth_path)
                    print(f"✓ Fold {fold_idx}: Aggiunto {pth_path}")
                
                # Carica artifact del fold
                run.log_artifact(artifact).wait()
                print(f"📦 Fold {fold_idx}: Artifact caricato!")
    
    # Carica summary come artifact separato
    if os.path.exists('cv_summary.csv'):
        summary_artifact = wandb.Artifact('cv_summary', type='summary')
        summary_artifact.add_file('cv_summary.csv')
        run.log_artifact(summary_artifact)
        print("✓ Summary caricato!")
    
    wandb.finish()  # ✅ Chiudi lo STESSO run
    print("\n✅ Tutti gli artifact caricati su wandb!")
    return True

print("✓ Funzione cross_validation definita")

In [ ]:
# Cella 6: Esecuzione training
if __name__ == "__main__":
    success = cross_validation()
    
    if success:
        print("\n🎉 Training completato con successo!")
        print("\n📁 File generati per ogni fold:")
        for fold_idx in range(NUM_FOLDS):
            print(f"  - fold{fold_idx}_best.pt (modello)")
            print(f"  - fold{fold_idx}_results.pt (metriche)")
            print(f"  - fold{fold_idx}_results.csv (training log)")
        print("\n📁 File aggregati:")
        print("  - cv_summary.csv (statistiche cross-validation)")
    else:
        print("\n❌ Training fallito!")

In [ ]:
# Cella 7 (BONUS): Visualizzazione risultati
def visualize_results():
    """Visualizza i risultati del training"""
    import matplotlib.pyplot as plt
    
    # Carica summary
    if not os.path.exists('cv_summary.csv'):
        print("❌ File cv_summary.csv non trovato")
        return
    
    summary = pd.read_csv('cv_summary.csv')
    
    # Plot metriche per fold
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    metrics = ['precision', 'recall', 'mAP50', 'mAP50-95']
    titles = ['Precision', 'Recall', 'mAP@0.5', 'mAP@0.5:0.95']
    
    for ax, metric, title in zip(axes.flat, metrics, titles):
        values = summary[metric].values
        folds = summary['fold'].values
        
        ax.bar(folds, values, color='steelblue', alpha=0.7)
        ax.axhline(values.mean(), color='red', linestyle='--',
                   label=f'Mean: {values.mean():.4f}')
        ax.set_xlabel('Fold')
        ax.set_ylabel(title)
        ax.set_title(f'{title} per Fold')
        ax.legend()
        ax.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('cv_metrics.png', dpi=150)
    plt.show()
    
    print("\n✓ Grafico salvato come cv_metrics.png")
    
    # Plot training curves per un fold
    fold_to_plot = 0
    csv_path = f'fold{fold_to_plot}_results.csv'
    
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # Loss
        ax = axes[0]
        if 'train/total_loss' in df.columns:
            ax.plot(df['epoch'], df['train/total_loss'], label='Train Loss', marker='o')
        if 'val/total_loss' in df.columns:
            ax.plot(df['epoch'], df['val/total_loss'], label='Val Loss', marker='s')
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Loss')
        ax.set_title(f'Fold {fold_to_plot} - Training & Validation Loss')
        ax.legend()
        ax.grid(alpha=0.3)
        
        # Metriche
        ax = axes[1]
        metric_cols = [col for col in df.columns if col.startswith('metrics/')]
        for col in metric_cols:
            values = df[col].dropna()
            if len(values) > 0:
                epochs = df.loc[values.index, 'epoch']
                label = col.replace('metrics/', '').replace('(B)', '')
                ax.plot(epochs, values, label=label, marker='o')
        
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Metric Value')
        ax.set_title(f'Fold {fold_to_plot} - Metrics')
        ax.legend()
        ax.grid(alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(f'fold{fold_to_plot}_curves.png', dpi=150)
        plt.show()
        
        print(f"✓ Grafico fold {fold_to_plot} salvato come fold{fold_to_plot}_curves.png")

# Visualizza i risultati
visualize_results()

print("✓ Funzione visualize_results definita")